In [101]:
import json
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import sys
import statsmodels.formula.api as smf
import pandas
import sklearn.datasets as sk_data
import sklearn.metrics as metrics
from sklearn.cluster import KMeans
import pandas as pd
from recsys.algorithm.factorize import SVD

import warnings
from sklearn.ensemble.forest import DecisionTreeRegressor as tree

import seaborn as sns
%matplotlib inline


### BUILD TREE AND TRAIN

#### 1. Type of Crime

In [90]:

daydict = {"monday":1, "tuesday":2, "wednesday":3, "thursday":4, "friday":5, "saturday":6, "sunday":7}

typeX = []
typey = []

inTypefd = open("../data/Boston-parse-type.dat.input","r")
for l in inTypefd.readlines():
    row = l.split("::")
    typeX.append([hash(row[0]),daydict[row[1].lower()]])
    typey.append(row[2][:-1])
inTypefd.close()

timeX = []
timey = []

inTimefd = open("../data/Boston-parse-time.dat.input","r")
for l in inTimefd.readlines():
    row = l.split("::")
    timeX.append([hash(row[0]),daydict[row[1].lower()]])
    timey.append(row[2][:-1])
inTimefd.close()

In [96]:
trainTypeSet = []

trainTypefd = open("../data/Boston-parse-type.dat.trainerr","r")
for l in trainTypefd.readlines():
    row = l.split("::")
    trainTypeSet.append(row)
trainTypefd.close()
print trainTypeSet[0]

['A1', 'Sunday', '41\n']


In [130]:

warnings.filterwarnings("ignore", category=DeprecationWarning) 


glerr = float('inf')
optLeaftype, optDepthtype = None, None
rate = 0.0

for leaf in xrange(22, 30):
    print 'leaf', leaf, optLeaftype, rate,":",
    """
    Predict with each model and find minimum error parameters
    """
    for depth in range(5, 16, 2):
        model = tree(min_samples_leaf=leaf, max_depth=depth)
        model = model.fit(typeX,typey)

        err = 0
        for l in trainTypeSet:            
            #print l
            x = [hash(l[0]),daydict[l[1].lower()]]
            
            pred = model.predict(x)[0]
            #print pred, l[2][:-1]
            #sys.exit(0)
            err += abs(pred-int(l[2][:-1]))            
        rate = 1.0*err/10000
        if rate < glerr:
            glerr = rate
            """
            Later use these to build predicting model:
            """
            optLeaftype, optDepthtype  = leaf, depth
        
    print glerr

leaf 22 None 0.0 : 198.823893647
leaf 23 22 198.825673121 : 198.823893647
leaf 24 22 198.825673121 : 198.823893647
leaf 25 22 198.825673121 : 198.823893647
leaf 26 22 198.825673121 : 198.823893647
leaf 27 22 198.825673121 : 198.823893647
leaf 28 22 198.825673121 : 198.823893647
leaf 29 22 198.825673121 : 198.823893647


In [131]:
print optLeaftype, optDepthtype

22 7


In [118]:
trainTimeSet = []

trainTimefd = open("../data/Boston-parse-time.dat.trainerr","r")
for l in trainTimefd.readlines():
    row = l.split("::")
    trainTimeSet.append(row)
trainTimefd.close()
print trainTimeSet[0]

['A1', 'Sunday', '2\n']


In [121]:

warnings.filterwarnings("ignore", category=DeprecationWarning) 


glerr = float('inf')
optLeaftime, optDepthtime = None, None
rate = 0.0

for leaf in xrange(22, 30):
    print 'leaf', leaf, optLeaftime, rate,":",
    """
    Predict with each model and find minimum error parameters
    """
    for depth in range(5, 16, 2):
        model = tree(min_samples_leaf=leaf, max_depth=depth)
        model = model.fit(timeX,timey)

        err = 0
        for l in trainTimeSet:                        
            x = [hash(l[0]),daydict[l[1].lower()]]
            
            pred = round(model.predict(x)[0])
            #print pred, l[2][:-1]
            #sys.exit(0)
            err += abs(pred-int(l[2][:-1]))            
        rate = 1.0*err/10000
        if rate < glerr:
            glerr = rate
            """
            Later use these to build predicting model:
            """
            optLeaftime, optDepthtime  = leaf, depth
        
    print glerr

leaf 22 None 0.0 : 4.4648
leaf 23 22 4.4648 : 4.4648
leaf 24 22 4.4648 : 4.4648
leaf 25 22 4.4648 : 4.4648
leaf 26 22 4.4648 : 4.4648
leaf 27 22 4.4648 : 4.4648
leaf 28 22 4.4648 : 4.4648
leaf 29 22 4.4648 : 4.4648


In [122]:
print optLeaftime, optDepthtime

22 5


### PREDICT USING TREES

#### 1. Type of Crime

In [135]:
model = tree(min_samples_leaf=optLeaftype, max_depth=optDepthtype)
model = model.fit(typeX,typey)

testTypefd = open("../data/Boston.dat.testpair","r")
resfd = open("../results/Boston-type-pred.dat","w")
for l in testTypefd.readlines():
    row = l.split("::")
    
    x = [hash(row[0]),daydict[row[1][:-1].lower()]]
    pred = round(model.predict(x)[0])
    resfd.write(row[0]+"-"+row[1][:-1]+"-"+str(pred)+"\n")
    #print pred
    #break
    
testTypefd.close()
resfd.close()

#### 2. Time of the day

In [138]:
model = tree(min_samples_leaf=optLeaftime, max_depth=optDepthtime)
model = model.fit(timeX,timey)

testTypefd = open("../data/Boston.dat.testpair","r")
resfd = open("../results/Boston-time-pred.dat","w")

for l in testTypefd.readlines():
    row = l.split("::")
    
    x = [hash(row[0]),daydict[row[1][:-1].lower()]]
    pred = model.predict(x)[0]*6
    resfd.write(row[0]+"-"+row[1][:-1]+"-"+str(pred)+"\n")
    #print pred
    #break
testTypefd.close()
resfd.close()

### BUILD MODEL AND TRAIN

In [60]:
svdK = 20
modelCrime = SVD()
modelCrime.load_data(filename='../data/Boston-parse-type.dat.input', sep='::', format={'col':0, 'row':1, 'value':2, 'ids': str})
modelCrime.compute(k=svdK, min_values=6, pre_normalize=None, mean_center=True, post_normalize=True)

In [70]:
trainTypefd = open("../data/Boston-parse-type.dat.trainerr","r")
for l in trainTypefd.readlines():
    row = l.split("::")
    try:        
        predTime = int(round(modelCrime.predict(row[1], row[0])%125))
        print row, predTime
        break
    except KeyError:
        print '.',
        continue

trainTypefd.close()

['A1', 'Sunday', '41\n'] 32


In [40]:
modelCrimeTime = SVD()
modelCrimeTime.load_data(filename='../data/Boston-parse-time.dat.input', sep='::', format={'col':0, 'row':1, 'value':2, 'ids': str})
modelCrimeTime.compute(k=svdK, min_values=6, pre_normalize=None, mean_center=True, post_normalize=True)

In [73]:
#../data/Boston-parse-time.dat.trainerr
trainTimefd = open("../data/Boston-parse-time.dat.trainerr","r")
for l in trainTimefd.readlines():
    row = l.split("::")
    try:
        predTime = str(round(modelCrimeTime.predict(row[1], row[0])%7))
        print row, predTime
        
        break
    except KeyError:
        print '.',
        continue


trainTimefd.close()


['A1', 'Sunday', '2\n'] 1.0


### GENERATE PREDICTION PAIRS

    Done during parsing phase

### PREDICTION

In [140]:
#../data/Boston-parse-time.dat.test
#D4:Sunday
#C6:Sunday
#E5:Sunday...
timePred = open("../results/recsys-Boston-time-pred.dat","w")
testTimefd = open("../data/Boston.dat.testpair","r")
for l in testTimefd.readlines():
    row = l.split("::")
    try:        
        predTime = str(modelCrimeTime.predict(row[1][:-1], row[0])%7)
        timePred.write(row[0]+"-"+row[1][:-1]+","+predTime+"\n")      
        #break
    except KeyError:
        #print row, '....',
        continue


testTimefd.close()
timePred.close()

In [141]:

typePred = open("../results/recsys-Boston-type-pred.dat","w")
typeTestfd = open("../data/Boston.dat.testpair","r")

for l in typeTestfd.readlines():
    row = l.split("::")
    try:        
        predType = str(modelCrime.predict(row[1][:-1], row[0])%125)
        typePred.write(row[0]+"-"+row[1][:-1]+","+predType+"\n")              
    except KeyError:
        #print row, '....',
        continue


typeTestfd.close()
typePred.close()

### ANALYSE PREDICTIONS (Error)

In [21]:
#rmse/classiff error?...